# Convolutional Neural Networks (CNN)

Convolutional neural networks (CNN) are powerful tools to perform image classification and segmentation. In this notebook, we explore the task of image classification using the CIFAR10 dataset.


In [ ]:
%matplotlib inline

In [109]:
import datetime
from pathlib import Path
from typing import Final, Literal, Iterator


import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as nnf
from torchvision import datasets, transforms

Load the data. The image normalization is taken from the web.

In [71]:
data_path = Path("../cifar_data")

class_names: list[str] = [
    'airplane','automobile','bird','cat','deer',
    'dog','frog','horse','ship','truck'
]

# Transform statistics taken from https://stackoverflow.com/a/69750247
cifar10_preprocessor = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]
)

cifar10_train = datasets.CIFAR10(
    data_path,
    train=True,
    download=True,
    transform=cifar10_preprocessor
)

cifar10_val = datasets.CIFAR10(
    data_path,
    train=False,
    download=True,
    transform=cifar10_preprocessor
)


Files already downloaded and verified
Files already downloaded and verified


Since I got the normalization values from the Internet, I should verify that these statistics are accurate. I will create numpy batch arrays and take the mean and std along the batch and 32x32 pixel axes.

In [36]:
# use np.concatenate to stick all the images together to form a (batch, 3, 32, 32) array
imgs = np.concatenate(
    np.asarray([[
        [
            cifar10_train[i][0][0].numpy(),
            cifar10_train[i][0][1].numpy(),
            cifar10_train[i][0][2].numpy()
        ]
        for i in range(len(cifar10_train))
    ]])
)

print(imgs.shape)

(50000, 3, 32, 32)


The array has the correct shape (batch, channels, height, width).

Since the image data was normalized, we should see zero mean and standard deviation of one (1) for the three channels.

In [39]:
# calculate the mean along the (batch, pixel, pixel) axes
train_mean = np.mean(imgs, axis=(0, 2, 3))
print(train_mean)

[-0.00040607 -0.0005815  -0.00102856]


In [40]:
# calculate the std along the (batch, pixel, pixel) axes
train_std = np.std(imgs, axis=(0, 2, 3))
print(train_std)


[1.0001289  0.9999368  0.99995327]


Great! The data is normalized with zero mean and standard deviation of one (1).

Now let's use a simplified model of distinguishing birds from airplanes. Let's call this CIFAR2

In [104]:
# New class names
cifar2_class_names: list[str] = ['airplane', 'bird']

# Convert between CIFAR10 and CIFAR2
cifar10_to_2_indices: list[int] = [
    class_names.index(cifar2) for cifar2 in cifar2_class_names
]
label_map: dict[int, int] = { #{0: 0, 2: 1}
    cifar2_ind: cifar10_ind
    for (cifar10_ind, cifar2_ind) in enumerate(cifar10_to_2_indices)
}

cifar2 = [
    (img, label_map[label])
    for img, label in cifar10_train
    if label in cifar10_to_2_indices
]

cifar2_val = [
    (img, label_map[label])
    for img, label in cifar10_val
    if label in cifar10_to_2_indices
]


One of important method to avoid overtraining is the use mini-batches selected in the training. Using the PyTorch DataLoader class provides this great functionality.

In [73]:
train_loader = DataLoader(cifar2, batch_size=64, shuffle=True)
val_loader = DataLoader(cifar2_val, batch_size=64, shuffle=False)

Depending on your host, you can use nVidia GPU granted it has CUDA support and installed.

In [87]:
device: Final = (
    torch.device('cuda') if torch.cuda.is_available()
    else torch.device('cpu')
)
device

device(type='cpu')

Let's implement the training loop

In [92]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    """Run training on a CNN

    Parameters
    ----------
    n_epochs : int
        Number of training iterations
    optimizer : optim.Optimizer
        Optimizer
    model : nn.Module
        CNN model
    loss_fn : nn.Module
        Loss function module
    train_loader : DataLoader
        Batched data loader

    Returns
    -------
    None
    """
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs_t, labels_t in train_loader:
            imgs_t = imgs_t.to(device=device)
            labels_t = labels_t.to(device=device)
            outputs = model(imgs_t)
            loss = loss_fn(outputs, labels_t)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        if epoch < 4 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

Now define the validation function

In [93]:
def validate(model, train_loader, val_loader):
    """Validate model training

    Parameters
    ----------
    model : nn.Module
        Trained model module
    train_loader : DataLoader
        Training data loading
    val_loader : DataLoader
        Validation data loader

    Returns
    -------
    dict[str, float]
        Accuracy dictionary for data loaders. The keys are ["train", "val"]
    """
    accdict: dict[Literal["train", "val"], float] = {}
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs_t, labels_t in loader:
                outputs = model(imgs_t)
                _, predicted_t = torch.max(outputs, dim=1)
                total += labels_t.shape[0]
                correct_t = predicted_t == labels_t
                correct += int(correct_t.sum())

        print("Accuracy {}: {:.2f}".format(name , correct / total))
        accdict[name] = correct / total
    return accdict

Let's define a small CNN that has 3 convolutional layers and two subsequent linear layers.

While rectified linear unit activation function is the practitioner favorite, let's continue using tanh and compare the results.

Let me include this important "gotcha" from the book. Consider some CNN model like below with the last layer with a log-softmax activation function and negative log-likelihood loss function.

```python
model = nn.Sequential(
            ...,
            nn.LogSoftmax(dim=1))
optimizer = nn.NLLLoss()
```

"_The combination of nn.LogSoftmax and nn.NLLLoss is equivalent to using nn.CrossEntropyLoss. This terminology is a particularity of PyTorch, as the nn.NLLoss computes, in fact, the cross entropy but with log probability predictions as inputs where nn.CrossEntropyLoss takes scores (sometimes called logits). Technically, nn.NLLLoss is the cross entropy between the Dirac distribution, putting all mass on the target, and the predicted distribution given by the log probability inputs._

_To add to the confusion, in information theory, up to normalization by sample size, this cross entropy can be interpreted as a negative log likelihood of the predicted distribution under the target distribution as an outcome. So both losses are the negative log likelihood of the model parameters given the data when our model predicts the (softmax-applied) probabilities. In this book, we won’t rely on these details, but don’t let the PyTorch naming confuse you when you see the terms used in the literature._

_It is quite common to drop the last nn.LogSoftmax layer from the network and use nn.CrossEntropyLoss as a loss._"

So let's keep this in mind for our application.

In [105]:
class Cifar2CNN(nn.Module):
    """CNN image classifier for two (2) classes"""

    def __init__(
        self,
        n_chans1=32
    ):
        """Instantiate a CNN CIFAR image classifier for two (2) classes.

        Parameters
        ----------
        n_chans1 : int
            Number of channels in the first layer

        """
        super().__init__()
        self.cifar_size: Final[int] = 32
        self.n_chans1: Final[int] = n_chans1
        self.cov_ker_size: Final[int] = 3
        self.cov_pad: Final[int] = 1
        # First convolutional layer (B, 3, 32, 32)
        self.conv1 = nn.Conv2d(
            3,
            n_chans1,
            self.cov_ker_size,
            padding=self.cov_pad
        )
        # Second convolutional layer, after applying pooling (B, n_chans1, 16, 16)
        self.conv2 = nn.Conv2d(
            n_chans1,
            n_chans1 // 2,
            self.cov_ker_size,
            padding=self.cov_pad
        )
        # Third convolutional layer, after applying pooling (B, n_chans1, 8, 8)
        self.conv3 = nn.Conv2d(
            n_chans1 // 2,
            n_chans1 // 2,
            self.cov_ker_size,
            padding=self.cov_pad
        )
        # Functional layer after convolutions and view/reshape (B, n_chans1 * 8 * 8, 32)
        self.fcn4 = nn.Linear(
            ((self.cifar_size // 4) ** 2) * (n_chans1 // 2),
            32
        )
        # Output functional layer after functional (B, 2)
        self.fcn5 = nn.Linear(32, 2)

    def forward(self, batch):
        """Propagate the batch forward through NN.

        Parameters
        ----------
        batch : torch.Tensor
            Batch of images

        Returns
        -------
        torch.Tensor
            Forward propagated tensor
        """
        out = nnf.max_pool2d(torch.tanh(self.conv1(batch)), 2)
        out = nnf.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, self.fcn4.in_features)
        out = torch.tanh(self.fcn4(out))
        out = self.fcn5(out)
        return out


Now let's try to train the model with a few epochs.

In [106]:
model = Cifar2CNN().to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 3,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

validate(model, train_loader, val_loader)

2022-12-24 11:00:24.718264 Epoch 1, Training loss 0.5437281327262805
2022-12-24 11:00:37.348607 Epoch 2, Training loss 0.45769688221299726
2022-12-24 11:00:53.493002 Epoch 3, Training loss 0.411329961126777
Accuracy train: 0.82
Accuracy val: 0.83


{'train': 0.8246, 'val': 0.8285}

Neat! Now we have a classifier.

Let's now demonstrate how we can use regularization to prevent overtraining. Using mini-batches already helps us, but this helps put us over-the-top. We could also explore other topics like Dropout.

In [122]:
def reg_lq(param, q_val):
    """Regularization additive value for loss/objective function

    Parameters
    ----------
    param : Iterator[Parameter]
        Iterator for model parameter tensors
    q_val : float
        Hyper-parameter value of the regularization power

    Returns
    -------
    torch.Tensor
        Single item tensor for regularization term
    """
    return sum(p.pow(q_val).sum() for p in param)


def reg_l2(param):
    """L2 ridge regularization term for a loss function"""
    return reg_lq(param, 2.0)


def reg_l1(param):
    """L2 euclidean regularization term for a loss function"""
    return reg_lq(param, 1.0)


def reg_elastic_net(param, scale):
    """Use elastic-net regularization as defined by 'Statistical Learning
    with Sparsity' by Hastie et al. where the hyperparameter scale is
    bounded [0, 1]"""
    assert 0 <= scale <= 1
    return 0.5 * (1.0 - scale) * reg_l2(param) + scale * reg_l1(param)


def training_loop_reg(
    n_epochs,
    optimizer,
    model,
    loss_fn,
    reg_fn,
    reg_scale,
    train_loader):
    """Run training on a CNN including regularization function

    Parameters
    ----------
    n_epochs : int
        Number of training iterations
    optimizer : optim.Optimizer
        Optimizer
    model : nn.Module
        CNN model
    loss_fn : nn.Module
        Loss function module
    reg_fn : Callable
        Regularization function
    reg_scale : float
        Regularization scale
    train_loader : DataLoader
        Batched data loader

    Returns
    -------
    None
    """
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs_t, labels_t in train_loader:
            imgs_t = imgs_t.to(device=device)
            labels_t = labels_t.to(device=device)
            outputs = model(imgs_t)
            loss = loss_fn(outputs, labels_t)
            reg = reg_fn(model.parameters())
            loss += reg_scale * reg
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        if epoch <= 5 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

In [119]:
model_l2 = Cifar2CNN().to(device=device)
optimizer_l2 = optim.SGD(model_l2.parameters(), lr=1e-2)

training_loop_reg(
    n_epochs = 5,
    optimizer = optimizer_l2,
    model = model_l2,
    loss_fn = nn.CrossEntropyLoss(),
    reg_fn = reg_l2,
    reg_scale = 1e-5,
    train_loader = train_loader,
)

validate(model_l2, train_loader, val_loader)

2022-12-24 12:42:39.395332 Epoch 1, Training loss 0.5681598976159551
2022-12-24 12:42:52.988805 Epoch 2, Training loss 0.4817252810213976
2022-12-24 12:43:05.774594 Epoch 3, Training loss 0.4442047304028918
2022-12-24 12:43:17.970108 Epoch 4, Training loss 0.40722736944058896
2022-12-24 12:43:29.057511 Epoch 5, Training loss 0.38068383248748294
Accuracy train: 0.83
Accuracy val: 0.84


{'train': 0.8343, 'val': 0.838}

In [123]:
model_l1 = Cifar2CNN().to(device=device)
optimizer_l1 = optim.SGD(model_l1.parameters(), lr=1e-2)

training_loop_reg(
    n_epochs = 5,
    optimizer = optimizer_l1,
    model = model_l1,
    loss_fn = nn.CrossEntropyLoss(),
    reg_fn = reg_l1,
    reg_scale = 1e-5,
    train_loader = train_loader,
)

validate(model_l1, train_loader, val_loader)

2022-12-24 12:50:57.417654 Epoch 1, Training loss 0.5543369795106778
2022-12-24 12:51:08.549444 Epoch 2, Training loss 0.4782435994619017
2022-12-24 12:51:20.202710 Epoch 3, Training loss 0.4442253862596621
2022-12-24 12:51:31.117058 Epoch 4, Training loss 0.40231277209937955
2022-12-24 12:51:46.275318 Epoch 5, Training loss 0.3692555131426283
Accuracy train: 0.84
Accuracy val: 0.85


{'train': 0.8413, 'val': 0.846}

In [124]:
model_elnet = Cifar2CNN().to(device=device)
optimizer_elnet = optim.SGD(model_elnet.parameters(), lr=1e-2)

training_loop_reg(
    n_epochs = 5,
    optimizer = optimizer_elnet,
    model = model_elnet,
    loss_fn = nn.CrossEntropyLoss(),
    reg_fn = lambda p: reg_elastic_net(p, 0.5),
    reg_scale = 1e-5,
    train_loader = train_loader,
)

validate(model_elnet, train_loader, val_loader)

2022-12-24 12:52:13.437420 Epoch 1, Training loss 0.5380308445851514
2022-12-24 12:52:24.351737 Epoch 2, Training loss 0.46329990731682746
2022-12-24 12:52:36.200076 Epoch 3, Training loss 0.4245817234182054
2022-12-24 12:52:48.165033 Epoch 4, Training loss 0.3874200144960622
2022-12-24 12:52:59.909681 Epoch 5, Training loss 0.3606618495693632
Accuracy train: 0.85
Accuracy val: 0.85


{'train': 0.8462, 'val': 0.8475}

If we had a more challenging problem than using binary classification like person recognition would be more interesting.